In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image

output_folder = "g_happy"
os.makedirs("images", exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

cuda = torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

latent_dim = 50
channels = 1


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 256, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        z = z.view(z.size(0), z.size(1), 1, 1)
        img = self.model(z)
        return img

generator = Generator()

model_fp = '/home/rnap/data/emotion/model/checkpoint_1000.tar'
checkpoint = torch.load(model_fp, map_location=device.type)
generator.load_state_dict(checkpoint['generator'])


if cuda:
    generator.cuda()

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

for i in range(1000):

    z = Tensor(np.random.normal(0, 1, (1, latent_dim)))

    gen_img = generator(z)
    image_filename = os.path.join(output_folder, "img{}.png".format(i))
    save_image(gen_img, image_filename, normalize=True)